## Install Required Packages

In [1]:
!pip install -q bitsandbytes datasets==2.21.0 peft==0.10.0 trl==0.8.6 distilabel==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm

## Load Tokenizer and Model

Remember to change model path for each of the three models!

In [3]:
from getpass import getpass
huggingface_token = getpass("Enter your Huggingface token:")

Enter your Huggingface token:··········


In [16]:
model_name = "ishasinha1/Qwen3-0.6B-SFT-Safety" # Remember to change this

device="cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            token=huggingface_token,
                                            quantization_config=BitsAndBytesConfig(
                                                  load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_use_double_quant=True,
                                                  bnb_4bit_quant_type='nf4'
                                              )
                                            ).to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.20M [00:00<?, ?B/s]

## Load MMLU Test Data

In [4]:
dataset = load_dataset("cais/mmlu", "all", split="test")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

## Helper functions for formatting questions and generating predictions

In [10]:
def format_question(ex):
    """
    Convert MMLU fields into a multiple-choice question.
    answer: index 0-3 (A-D)
    """
    choices = ["A", "B", "C", "D"]
    question = (
        f"{ex['question']}\n"
        f"A. {ex['choices'][0]}\n"
        f"B. {ex['choices'][1]}\n"
        f"C. {ex['choices'][2]}\n"
        f"D. {ex['choices'][3]}\n"
        "Answer:"
    )
    correct = choices[ex['answer']]
    return question, correct

In [11]:
max_new_tokens=1 # Only the correct answer (A-D) is needed

def predict_answer(question):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    for c in reversed(decoded):
        if c in ["A", "B", "C", "D"]:
            return c
    return None

## Evaluate Model Performance on MMLU

In [17]:
correct = 0
total = 0

for ex in tqdm(dataset):
    question, correct_ans = format_question(ex)
    pred = predict_answer(question)
    if pred == correct_ans:
        correct += 1
    total += 1

accuracy = correct / total * 100
print(f"\nMMLU Accuracy: {accuracy:.2f}% ({correct}/{total})")

100%|██████████| 14042/14042 [32:57<00:00,  7.10it/s]


MMLU Accuracy: 36.23% (5088/14042)
